<hr style="border:10px solid gray">

# Simple running example of DAHSI

<hr style="border:2px solid gray">

⚠️ In order to run this notebook, you need to have installed all the dependancies needed to run DAHSI as listed in the $\texttt{README.md}$ file in this repository.

<hr style="border:2px solid gray">

⚠️ This notebook is for illustration of the method purposes only and so the toy problem chosen contains no hidden variables to be able to go through the code in a few minutes. This notebook will give you the tools and understanding necessary to be able to build your own problem and solve it using DAHSI.

ℹ️ To check an example of hidden variables, go to the `Example_LorenzSynth` folder in the github repo and explore the differences between `File1.txt` and `File2.txt` shown in this tutorial and the ones found in that folder. It is the same problem but $y$ is hidden.

<hr style="border:10px solid gray">

## Mathematical background 

The algorithm data assimilation for hidden sparse inference (DAHSI) boils down to minimising the following cost function:

\begin{equation}
    \label{costfunk}
        A(\mathbf{X},\mathbf{p}) = \frac{1}{N}\sum_{i=1}^N \Vert \mathbf{X}(t_{i}) - \mathbf{Y}(t_{i}) \Vert^2 + \frac{1}{N}\sum_{i=1}^{N-1} R_f \left\{ \Vert \mathbf{X}(t_{i+1}) - \mathbf{f}(\mathbf{X}(t_i),\mathbf{p},\mathbf{\hat{F}}) \Vert^2 \right\} + \lambda \Vert \mathbf{p} \Vert_1.        
\end{equation}

his function is composed of three terms: the experimental error, $A_E(\mathbf{X},\mathbf{Y}) = \frac{1}{N}\sum_{i=1}^N \Vert \mathbf{X}(t_{i}) - \mathbf{Y}(t_{i}) \Vert^2$, the model error term, $A_M(\mathbf{X},\mathbf{p},\mathbf{\hat{F}}) = \frac{1}{N}\sum_{i=1}^{N-1} \left\{ \Vert \mathbf{X}(t_{i+1}) - \mathbf{f}(\mathbf{X}(t_i),\mathbf{p},\mathbf{\hat{F}}) \Vert^2 \right\}$, and a sparse penalty term $\lambda \Vert \mathbf{p} \Vert_1$. Here, $\mathbf{f}(\mathbf{X}(t_i),\mathbf{p},\mathbf{\hat{F}}) = \mathbf{X}(t_{i+1})$ defines the discrete time model dynamics and is obtained by discretizing the governing equations using a Hermite-Simpson collocation.

For full details on our method, check out our paper <a href="https://doi.org/10.1063/5.0066066">here</a>.

<hr style="border:10px solid gray">

We first import all llibraries we will use throughout the notebook.

In [ ]:
%matplotlib inline

import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cyipopt
from termcolor import colored
from tqdm.notebook import tqdm_notebook
import time
import sys
import pickle 
import os
import pyximport
pyximport.install()

<hr style="border:10px solid gray">

## Generating the data 

We will work with the Lorenz system as it is a classical example of chaotic systems.

We numerically simulate the system using Runge-Kutta 4th order with $\Delta t = 0.01$ to obtain the <i>data</i> we will use to show a simple running example. We will consider $N = 501$ time points, and we will add to the <i>data</i> some normally distributed noise with $\omega = 0.01$.

We will save the time-series for each variable in $\texttt{.dat}$ files.

In [ ]:
def RungeKutta4(f, t, y0, args=()):
    n = len(t)
    y = np.zeros((n, len(y0)))
    y[0] = y0
    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], y[i], *args)
        k2 = f(t[i] + h / 2., y[i] + k1 * h / 2., *args)
        k3 = f(t[i] + h / 2., y[i] + k2 * h / 2., *args)
        k4 = f(t[i] + h, y[i] + k3 * h, *args)
        y[i+1] = y[i] + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)
    return y

def Lorenz(t, y, sigma, rho, beta):
    return np.array([sigma * (y[1] - y[0]), 
                     y[0]*(rho - y[2]) - y[1], 
                     y[0]* y[1] - beta*y[2]])

y0 = [-8.0, 7.0, 27.0]
sigma = 10.0
rho = 28.0
beta = 8.0/3

dt = 0.01
N = 501
tfin = dt * (N - 1)
t = np.linspace(0, tfin, N)

sol = RungeKutta4(Lorenz, t, y0, args=(sigma,rho,beta))

# mean and standard deviation for the added Gaussian noise.
# let's make sure we always add the same noise instance.
np.random.seed(12345)
mu, sigma = 0, 0.01 
noise = np.random.normal(mu, sigma, size=(N, 3))

sol = sol + noise

x = sol[:,0]
y = sol[:,1]
z = sol[:,2]

xfile = open("datax_Lorenz.dat", "w")
yfile = open("datay_Lorenz.dat", "w")
zfile = open("dataz_Lorenz.dat", "w")

for i in range(N):
    xfile.write("%.5f\n" % x[i])
    yfile.write("%.5f\n" % y[i])
    zfile.write("%.5f\n" % z[i])

xfile.close()    
yfile.close()    
zfile.close()    

# plot the data we will use in DAHSI.
plt.rcParams["figure.figsize"] = (10,10)
plt.rcParams.update({'font.size': 18})
ax = plt.figure().add_subplot(projection='3d')

ax.plot(x, y, z, lw=2)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("Lorenz Attractor")

plt.show()    

<hr style="border:10px solid gray">

# A quick tour to the problem setup files

First we are going to look into how we define the generic equations, parameters, bounds etc. To do so, we will load `File1.txt` and go over each line.

In [ ]:
with open('File1.txt', 'r') as f:
    for i, line in enumerate(f, start=1):
        print(colored('%.2d','green') % i, '%s' % line.strip())        

The first line contains the number of state variables (`num_vars`), parameters (`num_params`) and measured state variables (`num_meas`), the time step $\Delta t$ and the total number of time points (`num_tpoints`), in that order and separated by commas. In the example file above, we can see that our problem has 3 state variables, 30 parameter to estimate and $3$ measured state variables. Our data consists of 501 time steps, with a step size of 0.01.
    
The next lines assign names to our state variables, each name in one different line. The observed variables are written first, and the hidden variables after. In this case we do not consider any hidden variables. In the example file above, the variables are named $x$, $y$ and $z$.

The next lines assign names to the parameters we seek to estimate, each name in one different line. In the example provided, the parameter names are $Bk0j$, for $k=1,2,3$, and $j=1,\dots,10$. 
    
The next lines define the equations of our problem. One line for each equation. The equations have to be written in the same order as the state variables. For the three variables, we consider a library of monomials of the three variables up to degree two.
    
We now set the upper and lower bounds for the state variables, separated by a comma. One line for each state variable.
    
Finally, we also get the upper and lower bounds for the parameters, separated by a comma. One line for each parameter.

Next we will look into `File2.txt`, were we define what our data files are.

In [ ]:
with open('File2.txt', 'r') as f:
    for i, line in enumerate(f, start=1):
        print(colored('%.2d','green') % i, '%s' % line.strip())     

This file contains the information about the measured state variables.
    
In the first lines we give names to the data of each measured state variable. One line for each measured state variable.
    
The next lines include the file name (and its extension) in which we can find the data of each measured variable. One line per measured variable, and in the same order as in the first part of the file.
    
In the example file above we have three measured state variables: we call them `datax` and `datay` and `dataz`; we then indicate that the data of our measured state variables can be found in `datax_Lorenz.dat`, `datay_Lorenz` and `dataz_Lorenz` files.

Finally, the variational annealing and model selection tuning parameters are defined in `File3.txt`.

In [ ]:
with open('File3.txt', 'r') as f:
    for i, line in enumerate(f, start=1):
        print(colored('%.2d','green') % i, '%s' % line.strip())     

The first two lines define values for $\alpha$ and $\beta_{\text{max}}$ for variational annealing. 
    
The next two lines indicate the initial $\lambda$ and the maximum value it can attain. This $\lambda$ value controls the sparsity of the models recovered. We should always start with one that includes all possible functions and set a maximum $\lambda$ for which all the terms drop to zero.
    
In the example file above we have set $\alpha = 2$ and a maximum $\beta$ value to 30. Initial $\lambda$ is set to $10^{-6}$ and the maximum to 68.

<hr style="border:10px solid gray">

# Compiling DAHSI

⚠️ It is **absolutely** necessary to run the following line of code. 

`compile.py` runs three crucial scripts that enable DAHSI to run:
* It first calculates the objective function, its Jacobian and Hessian for a general time point and saves them as strings. This is done via `Build_ObjJacHess.py`. The general expression for the objective function can be found in `Obj_Funks.py`;
* Then it runs `WriteStrings.py` which takes the strings generated in the first script and writes them in the "blank template" `OneLoopInC_Blank.pyx`, which will create the functions for the objective function, its Jacobian and Hessian in *cython* format. The output is the file `OneLoopInC.pyx`;
* Finally, it runs the file `setup.py` to create a `build` directory, a C file (`.c`), and a Shared Object file (`.so`). With this, we will be able to import our C-extension functions into our code.

In [ ]:
!python compile.py

Next, we load the variables needed from `ObjNeed.obj` created in `Build_ObjJacHess.py`. We only need the variables `row_final` and `col_final`, but by the nature of the `pickle` library, we need to load all the objects that were pickled into the file before the ones we need. 

In [ ]:
file_ObjJacHess = open('ObjJacHess.obj', 'rb') 

ObjFunk_Meas_eval = pickle.load(file_ObjJacHess)
ObjFunk_Model_eval = pickle.load(file_ObjJacHess)
Jacobian_Meas = pickle.load(file_ObjJacHess)
Jacobian_Model = pickle.load(file_ObjJacHess)
Hessian_Meas = pickle.load(file_ObjJacHess)
Hessian_Model = pickle.load(file_ObjJacHess)
row_final = pickle.load(file_ObjJacHess)
col_final = pickle.load(file_ObjJacHess)
nnzh = pickle.load(file_ObjJacHess)

We now can import all the modules and functions needed to run our code.

In [ ]:
# Read_Files.py reads the three input text files (File1.txt, File2.txt and File3.txt).
from Read_Files import *

# Obj_Funks defines the measured and model part of the action we are minimising.
from Obj_Funks import Meas_Funk
from Obj_Funks import Model_Funk

# We import the cost function, Jacobian and Hessian functions.
import OneLoopInC
from OneLoopInC import eval_f_tricky
from OneLoopInC import eval_grad_f_tricky
from OneLoopInC import eval_h_tricky

Finally we define a class (called DAHSI) that contains the objective function, the Jacobian and the Hessian, and define the constrains of the problem as empty.

In [ ]:
class DAHSI():
    def objective(self, x):
        """Returns the scalar value of the objective given x."""
        return eval_f_tricky(x,Rf)

    def gradient(self, x):
        """Returns the gradient of the objective with respect to x."""
        return np.transpose(eval_grad_f_tricky(x,Rf))

    def constraints(self, x):
        """Returns the constraints."""
        return array([ ], float_)

    def jacobian(self, x):
        """Returns the Jacobian of the constraints with respect to x."""
        return np.array([])

    # Location of the non-zero elements of the Hessian.
    def hessianstructure(self):
        """Returns the row and column indices for non-zero values of the
        Hessian."""
 
        return (np.array(col_final),np.array(row_final))

    def hessian(self, x, lagrange, obj_factor):
        """Returns the non-zero values of the Hessian."""        
        H = eval_h_tricky(x,lagrange,obj_factor,0,Rf)        

        row, col = self.hessianstructure()

        return H

<hr style="border:10px solid gray">

# Setting up the initial conditions

In [ ]:
# Choose seed for random number generation.
# We call this IC variable the taskID number.
IC = 0
np.random.seed(IC)

Use appropriate initial conditions: for observed state variables, the data provided; for hidden state variables, random; for parameters, set them all to 0.

In [ ]:
# Bounds for both state variables and parameters. 
x_L = np.ones((num_total))
x_U = np.ones((num_total))

for i in range(num_vars):
    x_L[i:-num_params:num_vars] = float(Input1[1+2*num_vars+num_params+i].split(",")[0])
    x_U[i:-num_params:num_vars] = float(Input1[1+2*num_vars+num_params+i].split(",")[1])
for i in range(num_params):
    x_L[-num_params+i] = float(Input1[1+3*num_vars+num_params+i].split(",")[0])
    x_U[-num_params+i] = float(Input1[1+3*num_vars+num_params+i].split(",")[1])
    
# Initial vector.    
x0 = (x_U-x_L)*np.random.rand(num_total)+x_L      
for i in range(num_meas):
    for k in range(0,num_vars*num_tpoints,num_vars):
        x0[k+i] = data[int(k/num_vars),i] 
for i in range(num_params):    
    x0[i-num_params] = 0
        
x_jp = np.zeros((num_total))    
for i in range(num_total):
    x_jp[i] = x0[i]    

Define the problem using cyipopt (the Python wrapper around Ipopt). We also can adjust some parameters for Ipopt iself.

In [ ]:
nlp = cyipopt.Problem(n = num_total, 
                      m = 0, 
                      problem_obj = DAHSI(), 
                      lb = x_L, 
                      ub = x_U, 
                      cl = np.array([]), 
                      cu = np.array([]))

# Change some options of the Ipopt solver.
nlp.add_option('linear_solver', 'ma97')
nlp.add_option('mu_strategy', 'adaptive')
nlp.add_option('adaptive_mu_globalization', 'never-monotone-mode')
nlp.add_option('bound_relax_factor', float(0))
nlp.add_option('print_level',0)

<hr style="border:10px solid gray">

# Running the $\lambda$ sweep

The core of the DAHSI algorithm is a nonlinear optimization step using VA, which is randomly initialized. At each VA step, we minimize $A_E+R_f A_M$ over state variable trajectories $\mathbf{X}(t)$ and parameters $\mathbf{p}$ given $R_f$ using IPOPT interfaced here via cyipopt.

Now we start the loop on $\lambda$. We start with a very small $R_f$ value and increase it as the VA step. We do this for every $\lambda$ we want to study.

In [ ]:
lambd = lambd_0

file_name = "D%s_M%s_IC%s_LorenzNotebook.dat" % (num_vars, num_meas, IC) 
file_results = os.path.join("outputfiles",file_name)
f = open(file_results,"w+")

print(colored('Variatonal annealing for different \lambda.', attrs=['bold']))
iter_count = 1
# Here starts the main loop
while lambd < lambd_max:   
    f = open(file_results,"a+")

    Rf0 = 1e-2

    for i in range(num_total):
        x_jp[i] = x0[i]    

    print("Iteration #%d: \lambda = %f"%(iter_count,lambd))
    for beta in tqdm_notebook(range(beta_max+1)):
        f = open(file_results,"a+")
        # Make note in results file which \lambda and \beta we are at.
        f.write("%f %f " % (lambd, beta))        

        # Controlling how much the model is enforced.
        Rf = Rf0*(alpha**beta)
  
        # Solve it via IPOPT (solution is x_jn).    
        x_jn, info = nlp.solve(x_jp)
            
        obj = info['obj_val']
        
        # We hard threshold the parameter part of the solution (the last num_params elements).
        for i in range(num_params):
            if abs(x_jn[i-num_params]) < lambd:
                x_jn[i-num_params] = 0

        # We set this solution as the initial condition for the next iteration of IPOPT. 
        x_jp = x_jn   

        # Write cost function value in file.
        f.write("%e " % obj)
                
        for k in range(num_params):
            f.write("%f " % x_jp[k-num_params])
        f.write("\n")     
        
        f.close()

    # Increase \lambda value.       
    lambd = 2*lambd
    iter_count = iter_count+1
f.close()   

num_lambda = iter_count-1

<hr style="border:10px solid gray">

# Basic analysis of the results

We first read the output file generated by the $\lambda$ sweep. This file is namedd `Di_Mj_ICk_LorenzNotebook.dat`, where `i` is the number of state variables, `j` is the number of measured state variables and `k` is the `taskID` we have chosen.
    
Each line in this file is the solution of the problem given a $\lambda$ and a $\beta$ value. For each $\lambda$, we will have $\beta_{\text{max}} + 1$ lines. So, if we do a sweep for $N_{\lambda}$ different $\lambda$ values, the file will have $N_{\lambda} (\beta_{\text{max}} + 1)$  lines.

In [ ]:
output_file = np.loadtxt('outputfiles/D3_M3_IC0_Lorenz.dat', unpack = False)

# Convert result file into lambda, active terms.
table_active_terms = np.zeros((num_lambda,2))
table_check_which = np.zeros((num_lambda,num_params))
for ll in range(num_lambda):
    active_terms = 0
    for i in range(num_params):
        if output_file[(ll+1)*beta_max+ll,3+i] != 0:
            active_terms = active_terms+1
            table_check_which[ll,i] = 1
    table_active_terms[ll,0] = output_file[(ll+1)*beta_max+ll,0]
    table_active_terms[ll,1] = active_terms

The action plot shown below shows how the action changes with increased $\beta$ values. We only show the action paths that correspond to the recovered correct model structure. We can observe that for high $\beta$ the model is fully enforced (i.e., the action plot plateaus), which is a good indicator that the algorithm has worked.

In [ ]:
for ll in range(num_lambda):    
    action_lambda = output_file[(ll)*beta_max+ll:(ll+1)*beta_max+ll,2]
    
    if table_active_terms[ll,1] == 7:
        plt.plot(action_lambda, marker='o', color="blue", linestyle='solid', markersize=12)
        plt.yscale("log")      
        plt.xlabel("$\\beta$")
        plt.ylabel("action")


## Models recovered

The number of active terms decreses as a function of $\lambda$, which shows us that the sparse model selection is working.

In [ ]:
for ll in range(num_lambda):   
    if table_active_terms[ll,1] == 7:
        plt.plot(table_active_terms[ll,0], table_active_terms[ll,1], marker='o', color="blue", linestyle='solid', markersize=17)
    else:
        plt.plot(table_active_terms[ll,0], table_active_terms[ll,1], marker='o', color="gray", linestyle='solid', markersize=17)
    
    plt.xscale("log")          
    plt.xlabel("$\lambda$")
    plt.ylabel("num. active terms")

In the following table we display the models recovered that contain 4, 7, 8 and 10 active terms. No models are found that contain 5, 6 or 9 active terms. This is because the we are doubling the $\lambda$ at every step and this is too coarse to capture all model complexities.

|   | Term | Parameter | 4 active terms | 7 active terms  | 9 active terms | 10 active terms
|-|-|-|-|-|-|-
| eq. $\dot{x}$ | 1 | $p_{1,1}$ |  | **--** | -- | x
| | $x$ | $p_{1,2}$ | x | **x** | x | x
| | $y$ | $p_{1,3}$ | x | **x** | x | x
| | $z$ | $p_{1,4}$ | -- | **--** | -- | --
| | $x^2$ | $p_{1,5}$ | -- | **--** | -- | --
| | $xy$ | $p_{1,6}$ | -- | **--** | -- | --
| | $xz$ | $p_{1,7}$ | -- | **--** | -- | --
| | $y^2$ | $p_{1,8}$ | -- | **--** | -- | --
| | $yz$ | $p_{1,9}$ | -- | **--** | -- | --
| | $z^2$ | $p_{1,10}$ | -- | **--** | -- | --
| eq. $\dot{y}$ | 1 | $p_{2,1}$ | -- | **--** | x | x
| | $x$ | $p_{2,2}$ | x | **x** | x | x
| | $y$ | $p_{2,3}$ | -- | **x** | x | x
| | $z$ | $p_{2,4}$ | -- | **--** | -- | --
| | $x^2$ | $p_{2,5}$ | -- | **--** | -- | --
| | $xy$ | $p_{2,6}$ | -- | **--** | -- | --
| | $xz$ | $p_{2,7}$ | -- | **x** | x | x
| | $y^2$ | $p_{2,8}$ | -- | **--** | -- | --
| | $yz$ | $p_{2,9}$ | -- | **--** | -- | --
| | $z^2$ | $p_{2,10}$ | -- | **--** | -- | --
| eq. $\dot{z}$ | 1 | $p_{3,1}$ | -- | **--** | x | x
| | $x$ | $p_{3,2}$ | -- | **--** | -- | --
| | $y$ | $p_{3,3}$ | -- | **--** | -- | --
| | $z$ | $p_{3,4}$ | x | **x** | x | x
| | $x^2$ | $p_{3,5}$ | -- | **--** | -- | --
| | $xy$ | $p_{3,6}$ | -- | **x** | x | x
| | $xz$ | $p_{3,7}$ | -- | **--** | -- | --
| | $y^2$ | $p_{3,8}$ | -- | **--** | -- | --
| | $yz$ | $p_{3,9}$ | -- | **--** | -- | --
| | $z^2$ | $p_{3,10}$ | -- | **--** | -- | --

The correct model is the model with 7 active terms (highlighted in bold in the table), and its equations are
\begin{align}
\dot{x} &= p_{1,2}x + p_{1,3}y,\\
\dot{y} &= p_{2,2}x + p_{2,3}y + p_{2,7}xz,\\
\dot{z} &= p_{3,4}z + p_{3,6}xy.
\end{align}

This is the structure of the Lorenz system, which means that we were able to recover the true system.